In [1]:
import pandas as pd
import numpy as np
import pydicom 
import os
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.models import Model, Sequential
from keras.layers import Concatenate, Dense, Input, concatenate
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, AveragePooling2D, Flatten

from sklearn.preprocessing import MinMaxScaler

In [2]:
def load_best_slices(path):
    ind = []
    for file in os.listdir(path):
        if file.split('.')[1] == 'txt':
            ind.append(file.split('.')[0])
        
    df = pd.DataFrame(index = ind, columns= ['CT'])
    for ind in df.index:
        df.loc[ind].CT = np.loadtxt(path + ind + '.txt')
    
    return df

df = load_best_slices('best_lung_slice/')

In [3]:
def custom_loss_function(original_array):
    original_array = original_array.astype('float32')

    def loss_function(y_true, y_pred):
        
        pred = un_scaler(y_pred, original_array)
        true = un_scaler(y_true, original_array)
        diff = abs(pred-true)
        
        return tf.reduce_mean(diff**4, axis=-1)**.25
    
    return loss_function

def custom_metric_function(original_array):
    original_array = original_array.astype('float32')

    def metric_function(y_true, y_pred):
        
        pred = un_scaler(y_pred, original_array)
        true = un_scaler(y_true, original_array)
        diff = abs(pred-true)
        diff = tf.where(diff > 7.5, 7.5, diff)
        diff = tf.where(diff < 0.5, 0.5, diff)
        
        return tf.reduce_mean(diff, axis=-1)
    
    return metric_function

In [4]:
df = df.reset_index()
df = df.rename(columns = {'index' : 'Patient'})

In [5]:
features = pd.read_csv('features.csv')
df = df.merge(features, on= 'Patient')

In [6]:
linear_data_all = pd.read_csv('patient_slope_intercept.csv', index_col=0)
result = pd.DataFrame(index = df.Patient, columns = ['slope'])
    
for ind in result.index:
    result.loc[ind].slope = linear_data_all.loc[ind].slope

df = df.merge(result, on='Patient')
df.iloc[:,2:] = df.iloc[:,2:].astype('float32')

In [7]:
#df.info()

In [8]:
dataset = df.values[:,1:]

In [9]:
scaler_features = MinMaxScaler()
dataset[:,1:-1] = scaler_features.fit_transform(dataset[:,1:-1])

In [10]:
def my_scaler(array):
    max_ = array.max()
    min_ = array.min()
    y = (array-min_)/(max_-min_)
    return 0.5*y + 0.25

def un_scaler(array, original_array):
    max_ = original_array.max()
    min_ = original_array.min()
    y = 2*(array - 0.25)
    return (max_-min_)*y + min_

#y_scaled = my_scaler(result)
# y = un_scaler(y_scaled, result)

In [11]:
scaled_dataset = np.copy(dataset)
scaled_dataset[:,-1] = my_scaler(scaled_dataset[:,-1])

#un_scaler(****, dataset[:,-1])

In [12]:
#just to set the values of the lung matrices between 0 and 1
scaled_dataset[:,0] = scaled_dataset[:,0]/10

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_dataset[:,:-1], scaled_dataset[:,-1], test_size=0.2)

In [14]:
lungs_train = []
for i in range(X_train[:,0].shape[0]):
    lungs_train.append(X_train[:,0][0].astype(np.float32))

lungs_train = np.array(lungs_train)    
lungs_train = lungs_train.reshape(X_train[:,0].shape[0], 512, 512, 1)

lungs_test = []
for i in range(X_test[:,0].shape[0]):
    lungs_test.append(X_test[:,0][0].astype(np.float32))

lungs_test = np.array(lungs_test)    
lungs_test = lungs_test.reshape(X_test[:,0].shape[0], 512, 512, 1)

In [15]:
features_train = []
for i in range(X_train[:,1:].shape[0]):
    features_train.append(X_train[i,1:].astype(np.float32))
    
features_test = []
for i in range(X_test[:,1:].shape[0]):
    features_test.append(X_test[i,1:].astype(np.float32))

In [16]:
features_train = np.array(np.array(features_train))
features_train = features_train.reshape(features_train.shape[0], 8)

features_test = np.array(np.array(features_test))
features_test = features_test.reshape(features_test.shape[0], 8)

In [ ]:
inp_conv = Input(shape = (512,512,1), name='lungs')

model_conv = Conv2D(16, kernel_size=(2,2), strides=(1,1), padding='valid', activation='relu')(inp_conv)
model_conv = Conv2D(16, kernel_size=(2,2), strides=(1,1), padding='valid', activation='relu')(model_conv)
model_conv = AveragePooling2D(pool_size=(3,3))(model_conv)
model_conv = Dropout(0.25)(model_conv)
model_conv = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu')(model_conv)
#model_conv = MaxPool2D(pool_size=(2,2))(model_conv)
model_conv = Conv2D(32, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu')(model_conv)
#model_conv = MaxPool2D(pool_size=(2,2))(model_conv)
model_conv = AveragePooling2D(pool_size=(3,3))(model_conv)
model_conv = Dropout(0.25)(model_conv)
model_conv = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu')(model_conv)
model_conv = Conv2D(64, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu')(model_conv)
model_conv = AveragePooling2D(pool_size=(3,3))(model_conv)
model_conv = MaxPool2D(pool_size=(2,2))(model_conv)
model_conv = Dropout(0.25)(model_conv)
model_conv = Flatten()(model_conv)
model_conv = Dense(16, activation='relu')(model_conv)
outp_conv = Dense(8, activation='sigmoid')(model_conv)

inp_feat = Input(shape = (8,), name='features')
model_feat = Dense(16, activation='relu')(inp_feat)
outp_feat = Dense(8, activation='relu')(model_feat)

model_conc = concatenate([outp_conv, outp_feat])
model_conc = Dense(32, activation='relu')(model_conc)
model_conc = Dense(16, activation='relu')(model_conc)
model_conc = Dense(8, activation='relu')(model_conc)
output = Dense(1, activation='sigmoid')(model_conc)

model = Model(inputs=[inp_conv, inp_feat], outputs=output, name="cnn_nn_model")

In [ ]:
model.summary()

In [ ]:
model.compile(loss=custom_loss_function(dataset[:,-1]), metrics=[custom_metric_function(dataset[:,-1])], optimizer='adam')

In [ ]:
model.fit(
    {"lungs": lungs_train, "features": features_train},
    y_train,
    epochs=2, #experimenta mudar este número
    batch_size=28,
    validation_data = ({"lungs": lungs_test, "features": features_test}, y_test)
)

In [ ]:
un_scaler(y_test.reshape(-1,1), dataset[:,-1])

In [ ]:
un_scaler(model.predict(([lungs_test, features_test])), dataset[:,-1])

In [ ]:
#diz-me que obtemos um número pequenino aqui para eu ficar feliz :D 
print(abs(un_scaler(model.predict(([lungs_test, features_test])), dataset[:,-1]) - un_scaler(y_test.reshape(-1,1), dataset[:,-1])).max())
print(abs(un_scaler(model.predict(([lungs_test, features_test])), dataset[:,-1]) - un_scaler(y_test.reshape(-1,1), dataset[:,-1])).min())
print(abs(un_scaler(model.predict(([lungs_test, features_test])), dataset[:,-1]) - un_scaler(y_test.reshape(-1,1), dataset[:,-1])).mean())

In [ ]:
un_scaler(model.predict(([lungs_test, features_test])), dataset[:,-1]) - un_scaler(y_test.reshape(-1,1), dataset[:,-1])

In [ ]:
print(abs(model.predict([lungs_test, features_test])  -y_test.reshape(-1,1)).max())
print(abs(model.predict([lungs_test, features_test])  -y_test.reshape(-1,1)).min())
print(abs(model.predict([lungs_test, features_test])  -y_test.reshape(-1,1)).mean())

In [ ]:
#last version

In [ ]:
tf.compat.v1.enable_eager_execution()

In [ ]:
tf.executing_eagerly() 

In [ ]:
tf.square(dataset[:,-1].astype('float32'))

In [ ]:
un_scaler(1.5,dataset[:,-1])